In [2]:
import openai
import instructor
from qdrant_client import QdrantClient

from pydantic import BaseModel, Field

### RAG Pipeline

In [3]:

client = instructor.from_openai(openai.OpenAI())

In [4]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [5]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "datamodel": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [6]:

qdrant_client = QdrantClient(url="http://localhost:6333")

In [11]:
output = rag_pipeline("Can I get a charging cable? Please suggest me a good one.", qdrant_client)

In [13]:
output

{'datamodel': RAGGenerationResponse(answer='Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.'),
 'answer': 'Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.',
 'question': 'Can I get a charging cable? Please suggest me a good one.',
 'retrieved_context_ids': ['B09PRK49JH',
  'B0BNHVLF7G',
  'B0CGM29ZL7',
  'B08BR9X387',
  'B0BB1YKXL1'],
 'retrieved_context': ['YUNZII AKKO Neon 157 Keycaps ASA Profile Keycap Set Double-Shot PBT Keycap Gift Set for Mechanical Keyboard (157 Keys, ASA NEON) 【Stand Out Aesthetic Color

In [12]:

print(output["answer"])

Yes, you can get a charging cable with the Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G). This product includes a Type-C quick charging cable along with the headphones. It is a good choice if you are looking for a high-quality sport headphone with a charging cable included.


### RAG Pipeline with Grounding Context

In [14]:
class RAGUsedContext(BaseModel):
    id: str = Field(description="The ID of the item used to answer the question")
    description: str = Field(description="Short description of the item used to answer the question")

class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    references: list[RAGUsedContext] = Field(description="List of items used to answer the question")

In [15]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain detailed information about the product and returned with detailed specification in bullet points.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "original_ouput": answer,
        "answer": answer.answer,
        "references": answer.references,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [16]:
result = rag_pipeline("Can I get some earphones?", qdrant_client, top_k=10)

In [17]:
result

{'original_ouput': RAGGenerationResponse(answer='Yes, you can get earphones from the available products. Here are two options:\n\n1. TELSOR Wireless Earbuds for iPhone:\n- Bluetooth 5.1 with stable transmission\n- Compatible with most Bluetooth-enabled devices including iPhones, Androids, smart TVs, and computers\n- Features 10mm diameter speaker and 2 microphones for clear calls and noise reduction\n- Deep bass for immersive music experience\n- Up to 6 hours playtime on a single charge, 30 hours with charging case\n- USB-C charging case, full charge in 2 hours\n- IPX7 waterproof rating\n- Ergonomic and lightweight design for comfortable wear\n- Smart touch controls for volume, playback, calls, and voice assistant\n\n2. TUNEAKE Kids Headphones:\n- Volume limited to 94dB for hearing protection\n- Padded headband and soft ear cups for comfort\n- Adjustable headband to fit kids\n- Folding design for easy storage and portability\n- Compatible with all devices with 3.5mm audio jack\n- No mi

In [18]:
print(result["answer"])

Yes, you can get earphones from the available products. Here are two options:

1. TELSOR Wireless Earbuds for iPhone:
- Bluetooth 5.1 with stable transmission
- Compatible with most Bluetooth-enabled devices including iPhones, Androids, smart TVs, and computers
- Features 10mm diameter speaker and 2 microphones for clear calls and noise reduction
- Deep bass for immersive music experience
- Up to 6 hours playtime on a single charge, 30 hours with charging case
- USB-C charging case, full charge in 2 hours
- IPX7 waterproof rating
- Ergonomic and lightweight design for comfortable wear
- Smart touch controls for volume, playback, calls, and voice assistant

2. TUNEAKE Kids Headphones:
- Volume limited to 94dB for hearing protection
- Padded headband and soft ear cups for comfort
- Adjustable headband to fit kids
- Folding design for easy storage and portability
- Compatible with all devices with 3.5mm audio jack
- No microphone included

These options cover wireless earbuds for general 